In [168]:

import pandas as pd
import numpy as np
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types

train = pd.read_csv(r"F:\DataScience\ml-scripting\study cases\titanic\datasets\train.csv")


In [169]:

train.drop(['PassengerId','Name','Ticket','Cabin','Fare'], axis=1, inplace=True)

X = train.drop('Survived', axis=1)
y = train['Survived']

display(X.head(4))
display(y.head(4))


,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S


In [172]:

X_evalml = infer_feature_types(X, feature_types={'Pclass': 'Categorical',
                                                 'Sex': 'Categorical',
                                                 'Age': 'Double',
                                                 'SibSp': 'Integer',
                                                 'Parch': 'Categorical',
                                                 'Embarked': 'Categorical'
                                                 })
X_evalml.ww


,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
Pclass,category,Categorical,['category']
Sex,category,Categorical,['category']
Age,float64,Double,['numeric']
SibSp,int64,Integer,['numeric']
Parch,category,Categorical,['category']
Embarked,category,Categorical,['category']


In [174]:

X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X_evalml, y, problem_type='binary', test_size=.2)


In [182]:

automl = AutoMLSearch(X_train=X_train,
                      y_train=y_train,
                      problem_type='binary',
                      objective='AUC',
                      max_batches=1)
automl.search()


Generating pipelines to search over...
8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: random_forest, xgboost, linear_model, extra_trees, catboost, lightgbm, decision_tree



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean AUC: 0.500

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + One Hot Encoder + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean AUC: 0.699
Decision Tree Classifier w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean AUC: 0.837
Random Forest Classifier w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean AUC: 0.858
LightGBM Classifier w/ Imputer + One Hot Encoder:
	Starting cross validation
	Finished cross validation - mean AUC: 0.849
Logistic Regression Classifier w/ Imputer + One Hot Encoder + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean AUC: 0.845
XGBoost Classifier w/ Imputer + One Hot Encod

In [180]:

automl.best_pipeline


pipeline = BinaryClassificationPipeline(component_graph=[Imputer, OneHotEncoder, ExtraTreesClassifier], parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'One Hot Encoder':{'top_n': 10, 'features_to_encode': None, 'categories': None, 'drop': 'if_binary', 'handle_unknown': 'ignore', 'handle_missing': 'error'}, 'Extra Trees Classifier':{'n_estimators': 100, 'max_features': 'auto', 'max_depth': 6, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1}}, custom_hyperparameters={'None':{'sampling_ratio': 0.25}}, random_seed=0)